# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings('ignore')

# Data

## Intake data

In [2]:
intake_url = "https://data.austintexas.gov/resource/wter-evkm.csv?$limit=500000"

In [3]:
df_intakes = pd.read_csv(intake_url)

## Outcome data

In [4]:
outcome_url = "https://data.austintexas.gov/resource/9t4d-g238.csv?$limit=500000"

In [5]:
df_outcomes = pd.read_csv(outcome_url)

# EDA

## Intake data

In [6]:
df_intakes.head()

,animal_id,name,datetime,datetime2,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color
0,A521520,Nina,2013-10-01T07:51:00.000,2013-10-01T07:51:00.000,Norht Ec in Austin (TX),Stray,Normal,Dog,Spayed Female,7 years,Border Terrier/Border Collie,White/Tan
1,A664235,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
2,A664236,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
3,A664237,NaN,2013-10-01T08:33:00.000,2013-10-01T08:33:00.000,Abia in Austin (TX),Stray,Normal,Cat,Unknown,1 week,Domestic Shorthair Mix,Orange/White
4,A664233,Stevie,2013-10-01T08:53:00.000,2013-10-01T08:53:00.000,7405 Springtime in Austin (TX),Stray,Injured,Dog,Intact Female,3 years,Pit Bull Mix,Blue/White


In [9]:
print(df_intakes['intake_type'].unique())

['Stray' 'Owner Surrender' 'Public Assist' 'Euthanasia Request' 'Wildlife'
 'Abandoned']


## Outcome data

In [7]:
df_outcomes.head()

,animal_id,date_of_birth,name,datetime,monthyear,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color
0,A668305,2012-12-01,NaN,2013-12-02T00:00:00-05:00,12-2013,Transfer,Partner,Other,Unknown,1 year,Turtle Mix,Brown/Yellow
1,A673335,2012-02-22,NaN,2014-02-22T00:00:00-05:00,02-2014,Euthanasia,Suffering,Other,Unknown,2 years,Raccoon,Black/Gray
2,A675999,2013-04-03,NaN,2014-04-07T00:00:00-05:00,04-2014,Transfer,Partner,Other,Unknown,1 year,Turtle Mix,Green
3,A679066,2014-04-16,NaN,2014-05-16T00:00:00-05:00,05-2014,NaN,NaN,Other,Unknown,4 weeks,Rabbit Sh,Brown
4,A680855,2014-05-25,NaN,2014-06-10T00:00:00-05:00,06-2014,Transfer,Partner,Bird,Unknown,2 weeks,Duck,Yellow/Black


In [11]:
print(df_outcomes['outcome_type'].unique())

# Value counts for outcome_type
print(df_outcomes['outcome_type'].value_counts())

['Transfer' 'Euthanasia' nan 'Adoption' 'Return to Owner' 'Died' 'Missing'
 'Disposal' 'Relocate' 'Rto-Adopt' 'Stolen' 'Lost']
outcome_type
Adoption           84598
Transfer           48689
Return to Owner    25691
Euthanasia         10833
Died                1672
Rto-Adopt           1241
Disposal             877
Missing               92
Relocate              29
Stolen                 5
Lost                   2
Name: count, dtype: int64


In [ ]:
# Clean column names
df_outcomes.columns = df_outcomes.columns.str.strip()

# Handle different column name formats
df_outcomes_ts = df_outcomes.rename(columns={
    'Animal Type': 'animal_type',
    'Outcome Type': 'outcome_type', 
    'MonthYear': 'monthyear'
})

# Group and count
aggregated_df_outcomes = (df_outcomes_ts.groupby(['animal_type', 'outcome_type', 'monthyear'])
                  .size()
                  .reset_index(name='count'))

# Rename and reorder columns
aggregated_df_outcomes = aggregated_df_outcomes.rename(columns={'monthyear': 'date'})
aggregated_df_outcomes = aggregated_df_outcomes[['count', 'animal_type', 'outcome_type', 'date']]

# Sort by date, then animal type
aggregated_df_outcomes = aggregated_df_outcomes.sort_values(['date', 'animal_type']).reset_index(drop=True)

In [ ]:
aggregated_df_outcomes.head(20)

,count,animal_type,outcome_type,date
0,159,Cat,Adoption,01-2014
1,1,Cat,Died,01-2014
2,23,Cat,Euthanasia,01-2014
3,38,Cat,Return to Owner,01-2014
4,146,Cat,Transfer,01-2014
5,359,Dog,Adoption,01-2014
6,3,Dog,Died,01-2014
7,45,Dog,Euthanasia,01-2014
8,1,Dog,Missing,01-2014
9,248,Dog,Return to Owner,01-2014
